# Day 13: Transparent Origami

In [1]:
from pathlib import Path
import re
from more_itertools import split_at
from dataclasses import dataclass
from functools import reduce, lru_cache

from aoc2021.util import read_as_list

## Puzzle input data

In [2]:
parse_line = lambda line: re.findall('[xy0-9]+', line.rstrip())

def split_input(lines: list):
    dots, folds = split_at(lines, lambda x: x == [])
    dots = [tuple(int(v) for v in d) for d in dots]
    folds = [tuple(int(v) if v.isdigit() else v for v in f) for f in folds]
    return dots, folds


# Test data.
tdata = split_input(map(parse_line, [
    '6,10',
    '0,14',
    '9,10',
    '0,3',
    '10,4',
    '4,11',
    '6,0',
    '6,12',
    '4,1',
    '0,13',
    '10,12',
    '3,4',
    '3,0',
    '8,4',
    '1,10',
    '2,14',
    '8,10',
    '9,0',
    '',
    'fold along y=7',
    'fold along x=5',
]))

# Input data.
data = split_input(read_as_list(Path('./day13-input.txt'), func=parse_line))
data[0][:5], data[1]

([(1053, 618), (966, 812), (363, 813), (663, 488), (582, 93)],
 [('x', 655),
  ('y', 447),
  ('x', 327),
  ('y', 223),
  ('x', 163),
  ('y', 111),
  ('x', 81),
  ('y', 55),
  ('x', 40),
  ('y', 27),
  ('y', 13),
  ('y', 6)])

## Puzzle answers
### Part 1

In [3]:
Dot = tuple[int,int]
Fold = tuple[str,int]
Input = tuple[list[Dot],list[Fold]]


@dataclass(frozen=True, eq=True)
class Paper:
    dots: set[Dot]

    @property
    @lru_cache(1)
    def size(self) -> tuple[int,int]:
        return self.width, self.height

    @property
    @lru_cache(1)
    def width(self) -> int:
        return max(x for x,_ in self.dots)
    
    @property
    @lru_cache(1)
    def height(self) -> int:
        return max(y for _,y in self.dots)

    @lru_cache(1)
    def __str__(self) -> str:
        return '\n'.join(''.join(['[▣]' if (c,r) in self.dots else ' . ' for c in range(self.width+1)]) for r in range(self.height+1))


def fold_coord(c: int, mag: int, dim: int):
    return max(dim,2*mag)-c if c >= mag else c+max(0,dim-2*mag)


def fold_dot(dot: Dot, fold: Fold, sz: tuple[int,int]):
    x, y = dot
    ax, mag = fold
    w, h = sz
    match ax:
        case 'x': x = fold_coord(x, mag, w)
        case 'y': y = fold_coord(y, mag, h)
        case _: raise Exception('invalid fold axis {ax}')
    return x, y


def fold_paper(pp: Paper, fold: Fold) -> Paper:
    dots = frozenset(fold_dot(d, fold, pp.size) for d in pp.dots)
    return Paper(dots)


def dots_visible(data: Input) -> Paper:
    dots, folds = data
    return fold_paper(Paper(frozenset(dots)), folds[0]).dots


assert Paper(frozenset(tdata[0])).width == 10
assert Paper(frozenset(tdata[0])).height == 14
assert Paper(frozenset(tdata[0])).size == (10, 14)
assert fold_dot((0,1),('y',6),(10,14)) == (0,3)
assert fold_dot((0,1),('y',10),(10,14)) == (0,1)
assert fold_dot((0,14),('y',6),(10,14)) == (0,0)
assert fold_dot((0,14),('y',10),(10,14)) == (0,6)
assert fold_dot((1,0),('x',4),(10,14)) == (3,0)
assert fold_dot((1,0),('x',7),(10,14)) == (1,0)
assert len(dots_visible(tdata)) == 17

In [4]:
n = len(dots_visible(data))
print(f'Number of dots visible after completing the first fold instruction on the transparent paper: {n}')

Number of dots visible after completing the first fold instruction on the transparent paper: 666


### Part 2

In [5]:
def ircamera_code(data: Input) -> Paper:
    dots, folds = data
    pp = Paper(frozenset(dots))
    return reduce(fold_paper, folds, pp)


assert len(ircamera_code(tdata).dots) == 16

In [6]:
code = ircamera_code(data)
print(f'Code to activate the infrared thermal imaging camera system: \n{code}')

Code to activate the infrared thermal imaging camera system: 
 . [▣][▣] .  .  .  . [▣][▣] . [▣] .  . [▣] .  . [▣][▣] .  . [▣][▣][▣][▣] . [▣] .  . [▣] . [▣] .  . [▣] . [▣] .  . [▣]
[▣] .  . [▣] .  .  .  . [▣] . [▣] .  . [▣] . [▣] .  . [▣] .  .  .  . [▣] . [▣] .  . [▣] . [▣] . [▣] .  . [▣] .  . [▣]
[▣] .  .  .  .  .  .  . [▣] . [▣][▣][▣][▣] . [▣] .  . [▣] .  .  . [▣] .  . [▣][▣][▣][▣] . [▣][▣] .  .  . [▣] .  . [▣]
[▣] .  .  .  .  .  .  . [▣] . [▣] .  . [▣] . [▣][▣][▣][▣] .  . [▣] .  .  . [▣] .  . [▣] . [▣] . [▣] .  . [▣] .  . [▣]
[▣] .  . [▣] . [▣] .  . [▣] . [▣] .  . [▣] . [▣] .  . [▣] . [▣] .  .  .  . [▣] .  . [▣] . [▣] . [▣] .  . [▣] .  . [▣]
 . [▣][▣] .  .  . [▣][▣] .  . [▣] .  . [▣] . [▣] .  . [▣] . [▣][▣][▣][▣] . [▣] .  . [▣] . [▣] .  . [▣] .  . [▣][▣] . 
